In [ ]:
import shutil

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split

from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

from lightautoml.addons.interpretation import LimeTextExplainer, L2XTextExplainer

import transformers
transformers.logging.set_verbosity(50)

import pickle

## Загружаем данные

Будем решать задачу классификации токсичности комментариев на датасете [Jigsaw-Toxic-Comment-Classification-Challage](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview).

In [ ]:
# train.csv file from
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/overview

data = pd.read_csv('train.csv')
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
np.random.seed(42)

train, test = train_test_split(data, test_size=0.2, random_state=42)

## AutoML часть

Мы будем дообучать TinyBERT из библиотеки [Hugging Face](https://huggingface.co) с помощью ``lightautoml.automl.presets.text_presets.TabularNLPAutoML`` пресета.

In [ ]:
roles = {
    'text': ['comment_text'], 
    'drop': ['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
    'target': 'toxic'
}

task = Task('binary')

# WAT

# automl = TabularNLPAutoML(
#     task=task,
#     autonlp_params={'model_name': 'wat',
#                     'transformer_params': {
#                         'weight_type': 'idf',
#                         'use_svd': True
#                    }}
# )


# BOREP

# automl = TabularNLPAutoML(
#     task=task,
#     autonlp_params={
#         'model_name': 'borep',
#         'transformer_params': {
#             'model_params':
#                 {'proj_size': 300, 'pooling': 'mean',
#                  'max_length': 200, 'init': 'orthogonal',
#                  'pos_encoding': False},
#         'dataset_params': {'max_length': 200}
#         }
#     }
# )


# Random LSTM

# automl = TabularNLPAutoML(
#     task=task,
#     autonlp_params={
#         'model_name': 'random_lstm',
#         'transformer_params': {
#             'model_params':
#                 {'embed_size': 300, 'hidden_size': 256,
#                  'pooling': 'mean', 'num_layers': 1},
#             'dataset_params': {'max_length': 200, 'embed_size': 300}
#         }
#     }
# )


automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='0',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']]
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'en',
        'bert_model': 'prajjwal1/bert-tiny'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7,
    }

)

In [ ]:
%%time
oof_pred = automl.fit_predict(train, roles=roles, verbose=10)
test_pred = automl.predict(test)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

print('Check scores:')
print('OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))
print('TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

[11:22:30] Stdout logging level is DEBUG.
[11:22:30] Model language mode: en
[11:22:30] Task: binary

[11:22:30] Start automl preset with listed constraints:
[11:22:30] - time: 3600.00 seconds
[11:22:30] - CPU: 1 cores
[11:22:30] - memory: 16 GB

[11:22:30] Train data shape: (127656, 8)

[11:22:30] Layer 1 train process start. Time left 3599.85 secs
[11:22:31] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[11:22:31] Training params: {'bs': 32, 'num_workers': 1, 'max_length': 128, 'opt_params': {'lr': 1e-05}, 'scheduler_params': {'patience': 5, 'factor': 0.5, 'verbose': True}, 'is_snap': False, 'snap_params': {'k': 1, 'early_stopping': True, 'patience': 1, 'swa': False}, 'init_bias': True, 'n_epochs': 7, 'input_bn': False, 'emb_dropout': 0.1, 'emb_ratio': 3, 'max_emb_size': 50, 'bert_name': 'prajjwal1/bert-tiny', 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': True, 'use_cat': True, 'use_text': True, 'lang': 'en', 'deterministic': False, 'multigpu': False, 'random_s

val: 100%|██████████| 1330/1330 [01:07<00:00, 19.83it/s]

[11:25:59] Epoch: 0, train loss: 0.25735557079315186, val loss: 0.19599375128746033, val metric: 0.9640350800072578



val: 100%|██████████| 1330/1330 [01:04<00:00, 20.58it/s]

[11:29:13] Epoch: 1, train loss: 0.16896754503250122, val loss: 0.14401142299175262, val metric: 0.9713461808486132


...

[12:31:15] number of text features: 1 
[12:31:15] number of categorical features: 0 
[12:31:15] number of continuous features: 0 


test: 100%|██████████| 998/998 [00:46<00:00, 21.51it/s]


[12:32:08] number of text features: 1 
[12:32:08] number of categorical features: 0 
[12:32:08] number of continuous features: 0 


test: 100%|██████████| 998/998 [00:46<00:00, 21.47it/s]


Check scores:
OOF score: 0.9782371823652668
TEST score: 0.9807740353486142
CPU times: user 18min 47s, sys: 1min 15s, total: 20min 3s
Wall time: 1h 10min 30s


In [ ]:
automl.set_verbosity_level(0) # refuse logging in automl

## LIME

![lime](https://github.com/sb-ai-lab/LightAutoML/blob/master/imgs/lime.jpg?raw=1)

In [ ]:
# LimeTextExplainer for NLP preset
lime = LimeTextExplainer(automl, feature_selection='lasso', force_order=False)

### Токсичные примеры

In [ ]:
exp = lime.explain_instance(test.loc[78687], labels=(0, 1), perturb_column='comment_text')
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 93.75it/s]


Text
 
 
 A silly fat cow who won ' t leave me alone 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.3 -1.8 -1.4 -0.9 -0.5 0.0 0.5 0.9 1.4 1.8 2.3 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.913

Еще один пример.

In [ ]:
exp = lime.explain_instance(test.loc[40112], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 56.36it/s]


Text
 
 
 stop editing this , you dumbass . why do you have to be such a bitch ? the ghosts of bill maas ' past will haunt you forever !!! MWAHAHHAHAA 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -2.8 -2.3 -1.7 -1.1 -0.6 0.0 0.6 1.1 1.7 2.3 2.8 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.920

Давайте заменим токсичные слова на 'good boy'

In [ ]:
test.loc[40112, 'comment_text'] = "stop editing this, you good boy. why do you have to be such a good boy? the ghosts of bill maas' past will haunt you forever!!! MWAHAHHAHAA"

In [ ]:
exp = lime.explain_instance(test.loc[40112], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 55.86it/s]


Text
 
 
 stop editing this , you good boy . why do you have to be such a good boy ? the ghosts of bill maas ' past will haunt you forever !!! MWAHAHHAHAA 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -1.5 -1.2 -0.9 -0.6 -0.3 0.0 0.3 0.6 0.9 1.2 1.5 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.157

Давайте попробуем из нейтрального текста сделать токсичный.

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 99.18it/s] 


Text
 
 
 Okay , thanks . I will do so . 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -1.2 -1.0 -0.7 -0.5 -0.2 0.0 0.2 0.5 0.7 1.0 1.2 
 

 Scale 
 ✕ 1e-03 

 AutoML's prediction 
 0.003

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 90.35it/s]


Text
 
 
 Okay , thanks . I will do so . dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -8.0 -6.4 -4.8 -3.2 -1.6 0.0 1.6 3.2 4.8 6.4 8.0 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.696

Добавим куда-нибудь позитивное слов.

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . happy dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:01<00:00, 85.69it/s]


Text
 
 
 Okay , thanks . I will do so . happy dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -7.0 -5.6 -4.2 -2.8 -1.4 0.0 1.4 2.8 4.2 5.6 7.0 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.513

Добавим больше позитивных слов.

In [ ]:
test.loc[18396] = "Okay , thanks . I will do so . happy cheerful joyfull glorious elated dumbass please"

In [ ]:
exp = lime.explain_instance(test.loc[18396], labels=(0, 1), perturb_column='comment_text', n_features=20) 
exp.visualize_in_notebook(1)

test: 100%|██████████| 157/157 [00:02<00:00, 74.52it/s]


Text
 
 
 Okay , thanks . I will do so . happy cheerful joyfull glorious elated dumbass please 
 
 
 
 Class mapping
 
 
 
 
 Class: 0
 
 Class: 1
 
 
 

 
 -4.9 -3.9 -2.9 -2.0 -1.0 0.0 1.0 2.0 2.9 3.9 4.9 
 

 Scale 
 ✕ 1e-01 

 AutoML's prediction 
 0.175

## L2X for Regression

Для L2X возьмем датасет [BeerAdvocate](http://people.csail.mit.edu/taolei/beer/). Он состоит из описаний алкогольных напитков и их пяти числовых характеристик. А качестве таргета будем использовать оценку внешнего вида.

In [ ]:
def download_from_gdrive(file_id, file_name, chunk_size=2**15):
    import requests
    
    def handle_warning(res):
        for k, v in res.cookies.items():
            if k.startswith("download_warning"):
                return v
            
    template_url = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    res = session.get(template_url, params={"id": file_id}, stream=True)
    print('GET: {} CODE'.format(res.status_code))
    token = handle_warning(res)
    if token:
        res = session.get(template_url, params={"id": file_id, "confirm": token}, stream=True)
    print('Started downloading...')
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            if chunk:
                f.write(chunk)
    print('Downloaded.')

    
download_from_gdrive('1s8PG13Y0BvYM67nNL0EQpdgB5S4gJK9r', 'beeradvocate.tar.gz')
shutil.unpack_archive('beeradvocate.tar.gz', '.')

GET: 200 CODE
Started downloading...
Downloaded.


In [ ]:
train_data = pd.read_csv('./datasets/reviews.aspect0.train.csv')
valid_data = pd.read_csv('./datasets/reviews.aspect0.heldout.csv')

train_data.head()

,Appearance,Aroma,Palate,Taste,Overall,Review,tokens_number
0,1.5,1.5,2.5,1.5,1.5,the main problem with this beer is that it has...,62
1,2.0,2.0,3.0,2.0,3.0,it is very unfortunate this situation we have ...,179
2,4.0,2.5,3.0,1.5,2.0,appearance is a light golden yellow with a thi...,79
3,4.5,3.5,2.0,3.5,3.0,it has a great color to the body . this beer p...,87
4,4.0,4.5,1.0,1.5,1.0,"though this beer is , or course , not carbonat...",246


### Train AutoML

In this part we use BERT-Base model.

In [ ]:
roles = {
    'text': ['Review'], 
    'drop': ['tokens_number', 'Aroma', 'Palete', 'Taste', 'Overall'],
    'target': 'Appearance'
}

task = Task('reg')

automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='1',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']],
        'n_folds': 3
    },
    reader_params={
        'cv': 3
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'en',
        'bert_model': 'bert-base-uncased'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7,
    },
)

oof_pred = automl.fit_predict(train_data, roles=roles, verbose=2)
test_pred = automl.predict(valid_data)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

print('Check scores:')
print('OOF score: {}'.format(mean_squared_error(train_data[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))
print('TEST score: {}'.format(mean_squared_error(valid_data[roles['target']].values, test_pred.data[:, 0])))

[12:38:00] Stdout logging level is INFO2.
[12:38:00] Task: reg

[12:38:00] Start automl preset with listed constraints:
[12:38:00] - time: 3600.00 seconds
[12:38:00] - CPU: 1 cores
[12:38:00] - memory: 16 GB

[12:38:00] Train data shape: (80000, 7)

[12:38:01] Layer 1 train process start. Time left 3599.63 secs
[12:38:01] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN ...
[12:38:01] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 834/834 [02:05<00:00,  6.64it/s]


[13:07:23] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_TorchNN =====


val: 100%|██████████| 834/834 [02:05<00:00,  6.66it/s]


[13:36:29] Time limit exceeded after calculating fold 1

[13:36:29] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN finished. score = -0.46728458911890136
[13:36:29] Lvl_0_Pipe_0_Mod_0_TorchNN fitting and predicting completed
[13:36:29] Time left 91.29 secs

[13:36:29] Time limit exceeded in one of the tasks. AutoML will blend level 1 models.

[13:36:29] Layer 1 training completed.

[13:36:29] Automl preset training completed in 3508.71 seconds

[13:36:29] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (2 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN) 



test: 100%|██████████| 313/313 [00:47<00:00,  6.64it/s]


Check scores:
OOF score: 0.46728458911890136
TEST score: 0.43322843977913716


In [ ]:
# >>> about 2gb
with open('apperance_model.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [ ]:
with open('apperance_model.pkl', 'rb') as f:
    automl = pickle.load(f)
automl.set_verbosity_level(2)

[13:38:29] Stdout logging level is INFO2.


## L2X

![GENERALL2X2](https://github.com/sb-ai-lab/LightAutoML/blob/master/imgs/GENERALL2X2.jpg?raw=1)

In [ ]:
l2x = L2XTextExplainer(automl, train_device='cuda:1',
                       inference_device='cuda:1',
                       embedding_dim=300,
                       gamma=0.1, temperature=2, temp_anneal_factor=0.95,
                       n_epochs=200, importance_sampler='gumbeltopk',
                       n_important=20, patience=25,
                       extreme_patience=30, trainable_embeds=True)

l2x.fit(train_data, valid_data, cols_to_explain='Review')

train nll (loss=0.1671): 100%|█████████▉| 1249/1251 [00:40<00:00, 30.58it/s]


In [ ]:
expl_valid = l2x['Review'].explain_instances(valid_data)

### Examples

In [ ]:
expl_valid[66].visualize_in_notebook()

Text
 
 
 <START> lot exploder lost about 3 of this beer down the drain as foam whats left is a cloudy medium brown color with floaties plenty of head obviously which dissipates quickly aroma is tons of malt and dark fruit the flavor is again very fruity with bready malt and caramel notes a bit of roast malt no hint of spices anywhere full bodied with plenty of silky crispness <PAD>

In [ ]:
expl_valid[77].visualize_in_notebook()

Text
 
 
 <START> the beer pours an opaque light copper capped by a minimal off white head there s very little retention despite a robust pour into the glass the nose is simply divine i feel like i just pulled a freshly baked pumpkin pie out of the oven aromas of gram cracker and butterscotch covered shortbread mix with sweet potato and canned <UNK> pumpkin puree coconut cinnamon and a hint of citrus add a twist of the exotic liquid pumpkin pie is the best way to describe the flavour a fine pte sucre crust with a rich pumpkin filling spiced with cinnamon and allspice this really is devilishly good everything i found lacking in previous pumpkin beers this makes up for smooth creamy macadamia nuttiness adds another dimension hops are nearly <UNK> simple there for balance similarly absent is the taste of alcohol despite the whopping percentage only int he very finish does it pop up like a <UNK> child the medium body and medium low level of carbonation make for a surprisingly easy drinking beer dangerous this is the best pumpkin beer i ve ever had hands down <PAD>

In [ ]:
expl_valid[111].visualize_in_notebook()

Text
 
 
 <START> dark black with creamy tan head that leaves great retention and foamy lace the smell is roasty with burnt sugar edges dark chocolate coffee and smoke the taste is ashy too much black patent perhaps others enjoy this but there is a charcoal burnt taste that is a bit much for me smoky bitter chocolate and roasted coffee quite roasty and ashy tasting strong with alcohol peeking through overall an average stout <PAD>

In [ ]:
expl_valid[9999].visualize_in_notebook()

Text
 
 
 <START> the apperance was an amber dark yellow color with not much head it did however have stuff floating in it i m not certain if that was of the fault of the manufacturer or the fault of myself for trusting my friends around my <UNK> drink any who the smell was bellow average although not always clearly present the taste was a sweet sour mix with a main taste of bitterness mouthfeel was smooth esque drinkability was average but seeing as i m a big time <UNK> and seeing as it s what my friends have i will most likely be having another very soon <PAD>

In [ ]:
expl_valid[7676].visualize_in_notebook()

Text
 
 
 <START> pale gold with a thin film around the edge some lacing looks very flat and insipid no carbonation i do n t hold out much hope very unpleasant sticky rice nose lots of nothingness as well sweet with a very light sickly note but do n t get me wrong it s incredibly bland blech thin but fortunately not overly sweet on the palate quite clean and dry with a light lingering bitterness mouthfeel is quite crisp which is a blessing no it s not great but i was expecting a lot lot worse it s really not that bad when you get down to it it s not amazing but it s pretty clean and light i guess i m just pleased it does n t have the sweet sickly character promised on the nose <PAD>

In [ ]:
expl_valid[6767].visualize_in_notebook()

Text
 
 
 <START> i was actually a little surprised by this one surprised it was not vile pours a clear gold color with a thin white head no real lacing to speak of and the head was short lived the aroma is lightly sweet which was another surprise light bodied with a barely average hops flavor the finish is a little sweet and a little fruity this is n t a beer i would seek out again but i would drink it in korea over a bud <PAD>

In [ ]:
expl_valid[3131].visualize_in_notebook()

Text
 
 
 <START> pours a clear deep red brown with a big white head malty sweet no major flavors stand out though it is slightly toasty hops are clean and mellow they only come in near the end and help to balance the beer this is a solid simple brown it s so great to finally see organic beer in the store <PAD>